<a href="https://colab.research.google.com/github/deniskapel/GenerativeChitChat/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
wget "https://raw.githubusercontent.com/deniskapel/GenerativeChitChat/master/requirements.txt"
pip install -r requirements.txt

In [2]:
import codecs
import json
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [ ]:
%%bash
mkdir data
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE" -O data/qa_data.jsonl.zip && rm -rf /tmp/cookies.txt
unzip data/qa_data.jsonl.zip -d data

In [41]:
assert torch.cuda.is_available(), 'no gpu available'
device = torch.device('cuda:1')

## Data

In [4]:
!sed 5q data/qa_data.jsonl

{"question": "долго ли идут деньги с яндексденег на карту visa?", "category": "Бизнес, Финансы", "responses": ["нет. прорыв 35 ;)"]}
{"question": "можно ли зарегистрировать авто в другом регионе", "category": "Авто, Мото", "responses": ["можно на родственника из того региона.. .  а потом ездить по доверке"]}
{"question": "что делать если у меня очень тонкие ногти а хочется их отрастить?", "category": "Красота и Здоровье", "responses": ["витамины и умная эмаль (каждый день)", "ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.", "умная эмаль, витамины, йод, и поменьше крась лаком ", "лаки фирмы trind производство usa + кальций"]}
{"question": "в чем отличие медитации от йоги?", "category": "Спорт", "responses": ["букв в йоге меньше", "в медитации ты просто сидишь и мммммычишь. а в йоге всяко разные упражнения вытворяешь", "в медитации вроде просто тупо сидишь и успокаеваешься, а в йоге еще и ноги за уши закидывать надо"]}
{"question": "когда нач

In [4]:
with codecs.open("data/qa_data.jsonl", encoding='utf-8-sig') as reader:
    lines = reader.read().split("\n")
    lines = list(map(json.loads, filter(None, lines)))

data = []
for line in tqdm(lines):
    for response in line['responses']:
        data.append(
            {'question': line['question'].lower(),
             'category': line['category'],
             'response': response.lower()})

del lines
df = pd.json_normalize(data)
del data

  0%|          | 0/2808811 [00:00<?, ?it/s]

In [ ]:
df.head()

In [28]:
df_mini = df.sample(frac=0.01, random_state=42)
train_df=df_mini.sample(frac=0.8,random_state=42)
test_df=df_mini.drop(train_df.index)

In [ ]:
train_df.category.value_counts()

In [30]:
tokenizer = yttm.BPE(model="pretrained/pretrained_bpe_lm.model")

In [31]:
train_df.question = tokenizer.encode(train_df.question.tolist(), bos=True, eos=True)
train_df.response = tokenizer.encode(train_df.response.tolist(), bos=True, eos=True)
test_df.question = tokenizer.encode(test_df.question.tolist(), bos=True, eos=True)
test_df.response = tokenizer.encode(test_df.response.tolist(), bos=True, eos=True)

In [44]:
train_df.question[0:1].tolist()

[[2, 1774, 2039, 16232, 4490, 3653, 1777, 10553, 3]]

In [53]:
q_lengths = np.array([len(x) for x in train_df.question.tolist()])
r_lenghts = np.array([len(x) for x in train_df.response.tolist()])
np.percentile(q_lengths, q=99), np.percentile(r_lenghts, q=99)

(33.0, 34.0)

In [54]:
batch_size = 64
q_max_len, r_max_len = 35, 35
pad_index = 0
eos_index = 3

In [60]:
class QAData(torch.utils.data.Dataset):
    
    def __init__(self, questions, responses, categories,
                 max_len_question, max_len_response,
                 pad_index, eos_index):
        
        self.x = questions
        self.y = responses
        self.categories = categories # to use with beam search later

        self.question_len = max_len_question
        self.response_len = max_len_response

        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        """ x is a question and y is an asnwer"""
        
        x = self.x[index][:self.question_len]
        y = self.y[index][:self.response_len]

        x_pads = [self.pad_index] * (self.question_len - len(x))
        y_pads = [self.pad_index] * (self.response_len - len(y))

        x = torch.tensor(x + x_pads).long()
        y = torch.tensor(y + y_pads).long()
        
        return x, y

In [61]:
train_dataset = QAData(
    questions=train_df.question.tolist(),
    responses=train_df.response.tolist(),
    categories=train_df.category.tolist(),
    max_len_question=q_max_len, max_len_response=r_max_len,
    pad_index=pad_index,
    eos_index=eos_index)

test_dataset = QAData(
    questions=test_df.question.tolist(),
    responses=test_df.response.tolist(),
    categories=test_df.category.tolist(),
    max_len_question=q_max_len, max_len_response=r_max_len,
    pad_index=pad_index,
    eos_index=eos_index)

len(train_dataset), len(test_dataset)

(62137, 15534)

In [62]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size)

In [69]:
for x, y in train_loader:
    break

x.shape, y.shape

(torch.Size([64, 35]), torch.Size([64, 35]))

## Model